In [1]:
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from numpy import argmax, array
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.linalg import pinv2
from sklearn.metrics import mean_squared_error
import warnings
import math
warnings.filterwarnings('ignore')

In [2]:
MAX_ITRERATION = 10
global input_weights
global bias
global g_best
global p_best
global g_best_x
global p_best_x

In [3]:
train = pd.read_csv("kdd_train.csv")
test = pd.read_csv("kdd_test.csv")

print(test.shape,train.shape)
#print(test['duration'])

(22544, 42) (125973, 42)


# Data Pre-Process


In [4]:
for i in ['protocol_type', 'service', "flag", "labels"]:
    values_train = array(train[i])
    values_test = array(test[i])
    label_encoder = LabelEncoder()
    integer_encoded_train = label_encoder.fit_transform(values_train)
    integer_encoded_test = label_encoder.fit_transform(values_test)
    train[i] = integer_encoded_train
    test[i] = integer_encoded_test

X_train = train.iloc[:, :41]
y_train = train.iloc[:, 41]

X_test = test.iloc[:, :41]
y_test = test.iloc[:, 41]

#print(X_train.head(),y_train.head())
#print(X_test.head(),y_test.head())
#print(list(y_train))#

In [5]:
rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=20)
# fit RFE
rfe.fit(X_train, y_train)
# summarize all features
rfe_X_train = pd.DataFrame()
rfe_X_test = pd.DataFrame()
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    col_name_train = train[train.columns[i]]
    col_name_test = test[test.columns[i]]
    if rfe.support_[i]:
        rfe_X_train = pd.concat([rfe_X_train, col_name_train], axis = 1)
        rfe_X_test = pd.concat([rfe_X_test, col_name_test], axis = 1)
      
print (rfe_X_train.shape, rfe_X_test.shape)
#print(col_name_train)

Column: 0, Selected True, Rank: 1.000
Column: 1, Selected True, Rank: 1.000
Column: 2, Selected True, Rank: 1.000
Column: 3, Selected True, Rank: 1.000
Column: 4, Selected True, Rank: 1.000
Column: 5, Selected True, Rank: 1.000
Column: 6, Selected False, Rank: 4.000
Column: 7, Selected False, Rank: 11.000
Column: 8, Selected False, Rank: 13.000
Column: 9, Selected True, Rank: 1.000
Column: 10, Selected False, Rank: 2.000
Column: 11, Selected False, Rank: 7.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected False, Rank: 10.000
Column: 14, Selected False, Rank: 15.000
Column: 15, Selected False, Rank: 16.000
Column: 16, Selected False, Rank: 8.000
Column: 17, Selected False, Rank: 9.000
Column: 18, Selected False, Rank: 18.000
Column: 19, Selected False, Rank: 21.000
Column: 20, Selected False, Rank: 22.000
Column: 21, Selected False, Rank: 19.000
Column: 22, Selected True, Rank: 1.000
Column: 23, Selected False, Rank: 6.000
Column: 24, Selected True, Rank: 1.000
Column: 25,

# PSO-ELM


In [27]:
input_size = rfe_X_train.shape[1]
hidden_size = 30


weights = np.array([np.random.uniform(-5,5, size=(input_size, hidden_size)) for i in range(25)])
bias = np.array([np.random.uniform(-5,5,hidden_size) for i in range(25)])
velocity_x = np.array([np.array([0 for j in range((input_size*hidden_size)+hidden_size)]) for i in range(25)])

input_weight_x = []

for i in range(25):
    input_weight_x.append(np.concatenate((weights[i].flatten(),bias[i])))

input_weight_x = array(input_weight_x)
print(input_weight_x.shape,bias)

(25, 630) [[-1.71438955  4.24321173 -0.70221343 -3.38114733  1.59024107  1.37902011
  -0.17101453 -1.31904073  1.78932785  4.05071167 -3.20453577  4.82866924
   3.30822788 -0.30779593 -2.20016388  2.99112718  1.00649762 -2.64776329
  -2.45683355 -3.87979988 -2.65437441  1.03808591 -3.7470437  -2.84659348
  -0.36501994 -3.4172665   0.01961334 -3.57550571  2.31151301  1.1958929 ]
 [-0.36868486 -4.70372524  1.29238803 -3.4145357   0.24143257 -3.73787257
   4.64537514  3.16299401  4.43435579 -3.38297696 -3.82837246 -1.77439351
  -4.32831218 -3.12667858  1.13969446 -3.95014242 -4.04443614 -1.35450605
  -2.09652334 -3.92237037 -4.96059255 -3.83152028 -2.05475114 -0.1234468
   1.80269141  1.71945067 -1.38629427  3.73092922  1.96903479  0.718721  ]
 [-1.46430064 -3.10252749 -1.78835004 -3.88179153 -1.0034729   1.55528406
  -4.73103983  1.27884977  2.61724618 -4.90308326  0.53690747  1.99385141
   0.67655108 -4.09136973  2.81585123 -4.98455147  3.86167359  2.76694381
   2.74902404  4.29560492 -

## ELM (Fitness Function)

In [28]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def hidden_nodes(X,input_weights,bias):
    G = np.dot(X, input_weights)
    G = G + bias
    H = sigmoid(G)
    return H

def predict(X,input_weights,output_weights,bias):
    out = hidden_nodes(X,input_weights,bias)
    out = np.dot(out, output_weights)
    return out

def ELM_fitness(row):
    input_weights = row[:(input_size*hidden_size)].reshape(input_size,hidden_size)
    bias = row[-hidden_size:]
    output_weights = np.dot(pinv2(hidden_nodes(rfe_X_train,input_weights,bias)), y_train)
    #pred
    prediction = predict(rfe_X_test,input_weights,output_weights,bias)

    return mean_squared_error(y_test,prediction)

In [50]:
p_best, p_best_x = [999 for row in input_weight_x], input_weight_x
g_best, g_best_x = 999,[]
print(p_best,p_best_x.shape)

[999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999, 999] (25, 630)


In [51]:
random_numbers = np.array([np.random.uniform(0,1,2) for i in range(30)])
print(random_numbers)

[[0.14187905 0.14158088]
 [0.19423788 0.36188961]
 [0.44357998 0.18585577]
 [0.53045679 0.16708379]
 [0.08532058 0.37810872]
 [0.11528206 0.40656828]
 [0.33018683 0.20427899]
 [0.06664516 0.48452822]
 [0.17588886 0.75303205]
 [0.22212796 0.06222223]
 [0.97889795 0.16673115]
 [0.72041394 0.54005196]
 [0.33960854 0.28737111]
 [0.72099115 0.31421369]
 [0.471728   0.18378947]
 [0.4814248  0.06069019]
 [0.28054827 0.25314772]
 [0.5844566  0.24913173]
 [0.93674045 0.54067596]
 [0.0739223  0.08512094]
 [0.15930631 0.66303691]
 [0.49320477 0.2921483 ]
 [0.50346092 0.17925975]
 [0.54767606 0.59170514]
 [0.67763138 0.78493512]
 [0.5494428  0.51676766]
 [0.7409649  0.2017191 ]
 [0.27713128 0.05406928]
 [0.62902519 0.36110205]
 [0.44960733 0.3826797 ]]


## Velocity Calculation

In [52]:
c1 = 1
c2 = 2
w = 0.6
def calculate_v(old_v,c1,c2,r1,r2,x,p_best_x,g_best_x):
  return w*old_v + c1*r1*(p_best_x-x) + c2*r2*(g_best_x-x)

In [53]:
for iteration in range(0,30):
    input_weight_x = input_weight_x + velocity_x
    current_x = [ELM_fitness(row) for row in input_weight_x]
    for i,f_val in enumerate(current_x):
        if p_best[i] > f_val:
            p_best[i] = f_val
            p_best_x[i] = input_weight_x[i]
        if p_best[i] < g_best:
            g_best = p_best[i]
            g_best_x = p_best_x[i]
        v = []
    print("Done with: ",iteration)
    for i, particle in enumerate(input_weight_x):
        v.append(calculate_v(velocity_x[i],c1,c2,random_numbers[iteration][0], random_numbers[iteration][1],particle,p_best_x[i],g_best_x))
    velocity_x = np.array(v)

Done with:  0
Done with:  1
Done with:  2
Done with:  3
Done with:  4
Done with:  5
Done with:  6
Done with:  7
Done with:  8
Done with:  9
Done with:  10
Done with:  11
Done with:  12
Done with:  13
Done with:  14
Done with:  15
Done with:  16
Done with:  17
Done with:  18
Done with:  19
Done with:  20
Done with:  21
Done with:  22
Done with:  23
Done with:  24
Done with:  25
Done with:  26
Done with:  27
Done with:  28
Done with:  29


In [54]:
#print(g_best_x.shape,g_best_x)
print(g_best)
print(math.sqrt(g_best))

42.072745576895805
6.486350713374648


# Best Input-Weight And Bias

In [117]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def hidden_nodes(X):
    G = np.dot(X, input_weights)
    G = G + bias
    H = sigmoid(G)
    return H

input_weights = g_best_x[:400].reshape(input_size,hidden_size)
bias = g_best_x[-hidden_size:]
output_weights = np.dot(pinv2(hidden_nodes(rfe_X_train)), y_train)
print(output_weights.shape)

(20,)
<ipython-input-117-13c3d2f2b0f1>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


In [118]:
def predict(X):
    out = hidden_nodes(X)
    out = np.dot(out, output_weights)
    return out

prediction = predict(rfe_X_test)
mean_squared_error(y_test,prediction)

<ipython-input-117-13c3d2f2b0f1>:2: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-x))


44.19176599137174

## Experimental Results
* Hidden-Neurons = 10
    * ELM(MSE = 48.14757009978859,RMSE = 6.9388450119446095)

    * PSO-ELM(MSE=44.802622846415446,RMSE=6.6934761407220575)

* Hidden-Neurons = 20
    * ELM(MSE = 47.31095358876977, RMSE = 6.878295834635915)
    * PSO-ELM(MSE=43.867156139935254,RMSE=6.62322852843953)
    
* Hidden-Neurons = 30
    * ELM(MSE = 46.12265073429617,RMSE=6.791365896069522)
    
    * PSO-ELM(MSE=42.39817268089611,RMSE=6.511387922777763)

## PSO-ELM Results
    * 10 - Iterations (MSE="42.39817268089611", RMSE="6.511387922777763") 
    * 20 - Iterations (MSE="42.09405068660949", RMSE="6.4879928087667835")
    * 30 - Iterations (MSE="42.072745576895805", RMSE="6.486350713374648")